# 基盤人工知能演習 第4回

※本演習資料の二次配布・再配布はお断り致します。

　今回の演習の内容は分類予測手法の1つである「**ロジスティック回帰 (Logistic regression)** 」についてである。

　**AI4.0 | データセットの作成**

　**AI4.1 | scikit-learnを用いたロジスティック回帰**

　**AI4.2 | ロジスティック回帰を実装する**

## AI4.0 | データセットの作成

　まずは、データセットの作成を行う。今回は、2つの説明変数 $x_1, x_2$ から、クラス0（負例, Negative）、クラス1（正例, Positive）を予測するための仮想的なデータを用いる（例えば、ロボットの2つのアームの角度を $x_1, x_2$ [rad] にしたときに、物体を掴むことができたらpositive、できなければnegative、などだろうか）。

　ここでは、**$-x_1 + 2x_2$の符号が最終的な結果を決める**こと、**結果がノイズで多少変化する**ことをイメージしながら、仮想的なデータを作成している。

In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [0]:
# 仮想的なデータの作成
np.random.seed(0)
n = 40
X_train = np.random.randn(n, 2)

noise = 1.6 * np.random.randn(n)                             # 結果にノイズが含まれている
y_train_bool = - X_train[:,0] + 2 * X_train[:,1] + noise > 0 # -x1 + 2 x2 > 0 をpositiveとして定義する
y_train = np.where(y_train_bool, 1, -1)                      # True -> 1, False -> -1 となるように変換

# 観測されたデータを散布図で示す
plt.scatter(X_train[:,0][y_train==1],X_train[:,1][y_train==1],  
            marker="o", label="positives")           # positive (y =  1) を o で表示
plt.scatter(X_train[:,0][y_train==-1],X_train[:,1][y_train==-1], 
            marker="x", label="negatives")           # negative (y = -1) を x で表示

# 真の境界線を表示
# y = - x_1 + 2 x_2 = 0 が境界線になるので x_1 = 2 x_2
plt.plot([-2, 2], [-1, 1], "gray", label="ground truth") # (-2, -1) から (2, 1) まで直線を引く

plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.legend(loc="lower left")
plt.show()

　正例と負例を直線で分離することは不可能だが、大まかには左上側に正例が、右下側に負例がまとまっているようである。

## AI4.1 | scikit-learnを用いたロジスティック回帰 (Logistic Regression)

　今回は、まず先にscikit-learnを用いてロジスティック回帰を実行してみる。ロジスティック回帰は、線形な分類モデルである（**回帰 (Regression) と名前がついているが、分類 (Classification) の手法であることに注意してほしい**）。



　scikit-learnにはロジスティック回帰は`LogisticRegression()`という名前で用意されているので、学習を行ってみる。計算内容の詳細は**AI4.2**節で説明するので、とりあえず利用してみよう。

In [0]:
from sklearn.linear_model import LogisticRegression

# 2019/12/13現在、デフォルトパラメータが変化しますよ、というFutureWarningが表示される
lr = LogisticRegression(fit_intercept=False) 
lr.fit(X_train, y_train)         # 学習の実行
print(lr.coef_)      # w_1, w_2 の値を出力

　$w = [-0.92, 1.46]$ という値が得られた。これは、 $-0.92x_1 + 1.46x_2 \ge 0$ であれば正例として、そうでなければ負例として予測する、という意味である。この条件から、**ロジスティック回帰の正例と負例の境界は直線になる**ことがわかる。この境界線を先ほど示したデータセットの散布図に重ね合わせてみる。

In [0]:
w_hat = lr.coef_[0]

In [0]:
# 観測されたデータを散布図で示す
plt.scatter(X_train[:,0][y_train==1],X_train[:,1][y_train==1],  
            marker="o", label="positives")           # positive (y =  1) を o で表示
plt.scatter(X_train[:,0][y_train==-1],X_train[:,1][y_train==-1], 
            marker="x", label="negatives")           # negative (y = -1) を x で表示

# 真の境界線を表示
# y = - x_1 + 2 x_2 = 0 が境界線になるので x_1 = 2 x_2
plt.plot([-2, 2], [-1, 1], "gray", label="ground truth") # (-2, -1) から (2, 1) まで直線を引く

# 予測された境界線を表示
# 境界線は必ず直線なので、2点だけ境界点を推定して、その間を直線で結んだ線分を描画する
# ax_1 + bx_2 = 0  <=>  x_2 = -(a x_1) / b 
xx1 = np.array([-2,2])
xx2 = -(w_hat[0] * xx1) / w_hat[1]
plt.plot(xx1, xx2, "red", label="estimated") 

plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.legend(loc="lower left")
plt.show()

　ロジスティック回帰では、**負例、正例に属する確率を`lr.predict_proba()`を用いて計算できる**。ここでは、$x_{new} = (2,0)$の正例、負例の確率を求めてみる。


In [0]:
print(lr.predict_proba([[2,0]]))

　86%の確率で負例、14%の確率で正例であるという結果が得られた（今回は負例、正例の順番に並んでいることに注意せよ）。散布図上の座標と見比べると、確かに $(2,0)$ はかなり負例側 (×側) の座標なので、予測結果は適切なようだ。

------
##### 課題 AI4.1

　以下のコードで作成された新しいデータ $X_{test}$, $y_{test}$ について、以下の問いに答えよ。

1. これまでのモデル`lr`を用いて、$X_{test}$から $\hat y_{test}$ を予測せよ。

2. 1.の予測結果と実際のラベル$y_{test}$を比較することで正解率 (Accuracy) を計算せよ。

In [0]:
## X_test, y_testの生成
np.random.seed(100)
n = 10
omega = np.random.randn()
noise = 1.6 * np.random.randn(n)
X_test = np.random.randn(n, 2)
y_test_bool = - X_test[:,0] + 2 * X_test[:,1] + noise > 0
y_test = np.where(y_test_bool, 1, -1)  # True/Falseを1/-1に変換
print(X_test)
print(y_test)

-----
##### 課題 AI4.2（発展、提出の必要はありません）

　**データに含まれるノイズが増えると予測は難しくなるはずである**。データ生成のコードの`noise`の`1.6`を適宜書き換え、正解率 (Accuracy) がどのように変化するか確認せよ。想定通り、予測問題の難易度は変化しているだろうか（ヒント：テストデータのサンプルサイズ $n$ を100などに増やした方が安定した結果が得られ、評価しやすくなる）。

------

## AI4.2 | ロジスティック回帰を実装する

　AI4.1で、ひとまずロジスティック回帰が利用できるようになった。ここから先は、どのように実装されているか、内部的な理解を深めていく。

### AI4.2.1 | 分類問題における損失関数（loss function）

　まず、基盤人工知能 第3回に習った回帰問題を思い出そう。回帰問題では、各予測結果に対して、実際の値 $y$ と予測した値 $y'$ の誤差（＝損失）を小さくするように学習しようと考えていた。この時に、二乗損失という損失関数を用いることで、 $w$ について**下に凸な関数**で、**1階微分が解析的に可能であり、微分値=0となるような $w$ を解析的に求められた**ため、 $L(w)$ が最小になるような $\hat w$ を陽に記述することが可能であった。



$\begin{eqnarray}
L(w)   & = & \frac{1}{2n} (y-Xw)^2 \\
\hat w & = & (X^TX)^{-1}X^Ty
\end{eqnarray}$


　分類問題の「誤差」を考えると、**実際の分類 $y$ と予測した分類 $y'$ が異なる確率が減少するように学習すれば分類問題を学習できる**はずである。

　これにL2正則化（前回のRidge回帰と同じ）を加えた損失関数 $L(w)$ で表現すると、以下のようになる（ **$y_i$ と $y'_i$ が同符号であれば分類が正解していて、異符号であれば分類が誤っていると判断**している）。

$\begin{eqnarray}
L(w) & = & \frac{1}{n} \sum^n_{i=1}[y_i y'_i < 0] + \lambda w^Tw \\
y'_i & = & w^T x_i \\
[cond ] & = & \left\{
    \begin{array}{l}
      1 (cond\text{ is True}) \\
      0 (cond\text{ is False})
    \end{array}
  \right.
\end{eqnarray}$



In [0]:
def predict(w, X):
  return np.dot(w.T, X.T)
def loss_func_1(w, X, y, lam):
  n = len(y)
  return 1/n * np.sum(y * predict(w, X) < 0 ) + lam * w.dot(w) 

　しかし、講義で確認した通り、これは **$w$ について下に凸な関数ではない**ため、**微分値=0**を条件としても最適な解を得ることができない。

In [0]:
## w2 = 1のまま、w1を変化させてみる
lam = 0.01
w2 = 1

loss_values = []
w1_values = np.arange(-3,3,0.01)
for w1 in w1_values:
  w = np.array([w1, w2])
  loss_values.append(loss_func_1(w, X_train, y_train, lam))
plt.plot(w1_values, loss_values, label="loss")
plt.legend()
plt.xlabel("w1")
plt.ylabel("loss")

　この理想的な損失関数は最適な解を求めることができないので、代わりにロジスティック損失を用いる。

$\begin{eqnarray}
L(w) & = & \frac{1}{n} \sum^n_{i=1}\log(1+\exp(-y_i y'_i)) + \lambda w^Tw \\
y'_i & = & w^T x_i
\end{eqnarray}$

In [0]:
def loss_func_2(w, X, y, lam):
  n = len(y)
  return 1/n * np.sum(np.log(1 + np.exp(-y * predict(w, X)))) + lam * w.dot(w) 

In [1]:
## w2 = 1のまま、w1を変化させてみる
lam = 0.01
w2 = 1

loss_values = []
w1_values = np.arange(-3,3,0.01)
for w1 in w1_values:
  w = np.array([w1, w2])
  loss_values.append(loss_func_2(w, X_train, y_train, lam)) #loss_func_2を計算
plt.plot(w1_values, loss_values, label="loss")
plt.plot(w2, loss_values, label="loss")
plt.legend()
plt.xlabel("w1")
plt.ylabel("loss")

NameError: name 'np' is not defined

　ロジスティック損失は、**下に凸な関数**になり、**解析的に $w$ に関する1階偏微分が求められる**。

$\begin{eqnarray}
\frac{\partial L(w)}{\partial w} = \frac{1}{n}\sum_{i=1}^n\frac{\exp(-y_i w^T x_i)(-y_i x_i)}{1+\exp(-y_i w^T x_i)} + 2\lambda w
\end{eqnarray}$

しかし、**$\frac{\partial L(w)}{\partial w} = 0$ を $w$ について解析的に解くことができない**ため、工夫が必要である。

--------------------

##### 課題 AI4.3

　`loss_func_2()`について、$w_2$ についてもグラフを描画することで、下に凸になっていることを確認せよ。また、$w_1=0$ である時、lossの値が最小になる $w_2$ の値を求めよ。小数点第2位を四捨五入して、小数点第1位まで答えよ（ヒント：`np.arange()`の範囲を狭めることで表示領域を限定し、目視で答えよ）。


--------------------

##### 課題 AI4.4（発展）

　以下の損失関数の実装を完成させ、これまでと同様の重み$w_1$と損失値とのグラフを描画することで、この損失関数は$w_1$について下に凸になっているかどうか推定せよ。なお、この損失関数は $y_i \cdot y_i' = 1 $ なる $i$ が存在するとき一般的な微分は不可能であることに注意せよ。

$\begin{eqnarray}
L(w) & = & \frac{1}{n} \sum^n_{i=1} l(y_i, y'_i) + \lambda w^Tw \\
y'_i & = & w^T x_i \\
l(y, y') & = & \left\{
    \begin{array}{l}
      -y \cdot y' + 1 \text{ (}y \cdot y' < 1\text{)} \\
      0 \text{ (}y \cdot y' \ge 1\text{)}
    \end{array}
  \right.
\end{eqnarray}$


In [0]:
###### l()を実装せよ ######
# y_true, y_predはともにnp.arrayであることに注意
def l(y_true, y_pred):
  return None
###########################

def predict(w, X):
  return np.dot(w.T, X.T)
def loss_func_exercise(w, X, y, lam):
  n = len(y)
  return 1/n * np.sum(l(y, predict(w, X))) + lam * w.dot(w) 

In [0]:
## w2 = 1のまま、w1を変化させてみる
lam = 0.01
w2 = 1

loss_values = []
w1_values = np.arange(-1,0,0.01)
for w1 in w1_values:
  w = np.array([w1, w2])
  loss_values.append(loss_func_exercise(w, X_train, y_train, lam)) 
plt.plot(w1_values, loss_values, label="loss")
plt.legend()
plt.xlabel("w1")
plt.ylabel("loss")

------

### AI4.2.2 | 最急降下法によるパラメータの最適化

　ロジスティック損失によって、下に凸な損失関数を定義することはできたが、この関数は損失関数の値が最小になるような $w$ の値を解析的に求めることはできなかった。
しかし、コンピュータを用いた計算の場合、求めたい値が解析的には求められない場合でも、**繰り返しの計算を行うことで、数値的に答えに近づくことが可能**である。
この節では、最も単純な**最急降下法 (steepest gradient descent)** を用いた $w$ の最適化を行う。

#### 最急降下法のイメージ

　まず、最急降下法のイメージを掴むために、$y=x^2$ について、 $y$ が最小になる $x$ を求めることを考えよう。
もし、関数が下に凸であれば、**関数の坂に玉を転がせば、いつかは $y$ の最小値にたどりつける**はずである（**図 AI4.1**）。これは、**坂を下る方向に最適な $x$ が存在する**という仮定を置いていることになる。

　最急降下法では、$\Delta t$ 時間後の玉の座標を、元いた座標における $y$ の勾配 $\left.\frac{dy}{dx}\right|_{x=x_0}$から推定する。

![図 AI4.1](https://i.imgur.com/tVHnJNE.png)

**図 AI4.1 | 最急降下法のイメージ**


#### 数値計算による最適化の枠組み

　数値的に最適な$w$を計算するためには、（仮想的な）時刻$t$を定め、以下のような更新式を繰り返し実行する。

$\begin{eqnarray}
w^{(t+1)} & = & w^{(t)} + \alpha^{(t)}d^{(t)}
\end{eqnarray}$

この式において、$d^{(t)}$は最適な $w$ が存在すると考えられる方向（ベクトル）であり、$\alpha^{(t)}$は時刻$t$における更新量（$\Delta t$ に相当）を定めている。

In [0]:
def optimize(X, y, lam, alpha, niter=1000):
  w = np.zeros(2) # 時刻t=0におけるwは (0,0) とする
  ww = [w]        # 後の描画のために履歴を残す
  
  for t in range(niter):
    w = w + alpha * direction(X, y, w, lam) # update (alphaは固定としている)
    ww.append(w)

  return np.array(ww) # 履歴を全て出力する

#### 最急降下法における更新幅 $\alpha^{(t)}$ と探索方向 $d^{(t)}$ 

　各最適化手法で異なるのは $\alpha^{(t)}$ と $d^{(t)}$ である。最急降下法においては $\alpha$ は $t$ によらず一定とすることが多いので、ここではそれに従う。

　最急降下法では、**関数の形を坂のようなものであると考え、ある点の勾配 (gradient) を計算して、坂を下る方向に最適解 $w$ がある**という仮説に基づき、 $d^{(t)}$ を決定する。
勾配とは、点$w^{(t)}$における最適化したい関数 $f(w)$（今回の場合は損失関数$L(w)$）の $w$ による1階偏微分 $\frac{\partial L(w)}{\partial w}$ に相当する。

　先ほども記述したが、ロジスティック損失 $L(w)$ は1階微分が可能なので、解析的に座標 $w$ における微分値を計算することが可能である（**補足資料 ※1**）。

$\begin{eqnarray}
\frac{\partial L(w)}{\partial w} = \frac{1}{n}\sum_{i=1}^n\frac{\exp(-y_i w^T x_i)(-y_i x_i)}{1+\exp(-y_i w^T x_i)} + 2\lambda w
\end{eqnarray}$

ただし、値の最小化を行うことから、勾配の負の方向に進むようにしよう。すなわち、 $d^{(t)} = -\frac{\partial L(w)}{\partial w}$ となる。

In [0]:
def predict(w, X):
  return np.dot(w.T, X.T)

def direction(X, y, w, lam):
  block = np.exp(-y * predict(w, X))
  numerator = block[:,np.newaxis] * (-y[:,np.newaxis] * X)
  denominator = 1 + block
  delta_w = 1 / len(y) * np.sum(numerator / denominator[:,np.newaxis], axis=0) + 2 * lam * w
  return - delta_w # 負の方向

　これで先ほど実装したoptimizeが動くはずだ。実行してみる。

In [0]:
## gradient discent
lam = 0.0125
alpha = 0.1

ww = optimize(X_train, y_train, lam, alpha)
print(ww)

　`optimize()`関数は、重みの変化を全て記録しているので、変化を描画することが可能である。

In [0]:
plt.plot(ww[:,0], ww[:,1], "o-", linewidth=0.5, markersize=3) # 丸マーカー付きの折れ線グラフ o-
plt.xlabel('$w_1$')
plt.ylabel('$w_2$')
plt.show()

　右下の $w=(0,0)$ を始点として、左上方向に $w$ が更新され、最終的にscikit-learnの結果と同じ $\hat w = (-0.92, 1.46)$ が得られた。また、最終的な解に近づくにつれ更新幅が減少していることもわかる。

------
##### 課題 AI4.5

　$\alpha$ の値を変更させることで、学習の進み方はどうなるだろうか。$\alpha = 0.001$ 、 $\alpha = 10$ 、およびその間の値を試すことで、**$\alpha$ が大きくなりすぎた時と $\alpha$ が小さくなりすぎた時のそれぞれについてどういうことが起きるか簡潔に述べよ。**

---------

# レポート提出について



## レポートの提出方法
　レポートは**答案テンプレートを用い**、**1つのファイル (.doc, .docx, .pdf)** にまとめ、**学籍番号と氏名を確認の上**、**2020/1/9 15:00 (次回 基盤人工知能演習) までに東工大ポータルのOCW-iから提出**すること。
ファイルのアップロード後、OCW-iで「提出済」というアイコンが表示されていることを必ず確認すること。それ以外の場合は未提出扱いとなるので十分注意すること。
また、締め切りを過ぎるとファイルの提出ができないため、時間に余裕を持って提出を行うこと。


## 答案テンプレート

```
学籍番号:
名前:

課題 AI4.1
1. 予測されたy：[__, __, __, __, __, __, __, __, __, __]
2. 正解率：

課題 AI4.3
w_2 = _____ の時、lossの値が最小になる

課題 AI4.4（発展）
この損失関数は w_1 について下に凸になって{いる | いない}。

課題 AI4.5
αが大きくなりすぎると__________________
αが小さくなりすぎると__________________

```


# 補足資料


## ※1 微分ができない場合の処理
微分ができない時は微分の定義から周囲に$w$の値を僅かにずらして、微分の定義に従って値を推定する。ただし、これを行うためには最低でも変数の数だけ微小にずらした座標の値の計算を行うため、計算的にはかなりコストが大きい。